# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [105]:
import warnings
warnings.filterwarnings('ignore')

In [106]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [107]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [108]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to l...
5,L'Or Espresso Café,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [109]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [110]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1)



- Low temperature (0 to 0.3): For accuracy and consistency in text generation
- Higher temperature (above 0.5): For creative or varied generations

Since we will be generating product descriptions (e.g. for an online store), we need some creativity and should work with a Temperature value around 1.

In [111]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "Write a funny online-store description of this product: {product}"
)

In [112]:
chain = prompt | llm

In [113]:
product = "cheese-shaped slippers" #Select a product type to be describe
result = chain.invoke(product)
print(result)

content="Introducing the most delectable footwear you'll ever slip your feet into - cheese-shaped slippers! Made with the finest faux fromage materials, these deliciously cheesy slippers are perfect for lounging around the house or indulging in a late-night snack attack. Say goodbye to boring old socks and hello to gourmet fashion for your feet. Just be careful, you may attract some hungry mice with these cheesy kicks! So, go ahead and treat yourself to a pair of cheese-shaped slippers - after all, who doesn't love a little extra cheese in their life?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 21, 'total_tokens': 136, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b1241485-6dbf-4dba-964e-17043d2386c4-0' u

## SimpleSequentialChain

In [114]:
from langchain.chains import SimpleSequentialChain

In [115]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write an online-store description of this product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [116]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this product description in 3 key bullet-points {review}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [117]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [118]:
for product in df['Product']:
    overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Treat yourself to the ultimate luxury with our Queen Size Sheet Set. Made from high-quality, ultra-soft materials, these sheets are designed to provide a peaceful night's sleep every time. The set includes a flat sheet, fitted sheet, and two pillowcases, all crafted with meticulous attention to detail for a perfect fit on your queen size bed. Available in a variety of colors and patterns to suit any decor style, our Queen Size Sheet Set is the perfect addition to your bedroom oasis. Upgrade your bedding experience today and experience the difference for yourself.
- Queen size sheet set made from high-quality, ultra-soft materials for a peaceful night's sleep
- Includes flat sheet, fitted sheet, and two pillowcases with perfect fit on queen size bed
- Available in variety of colors and patterns to suit any decor style, perfect addition to your bedroom oasis

> Finished chain.


> Entering new SimpleSequentialChain chain...
Keep your phone 

**Repeat the above twice for different products or reviews**

## SequentialChain

In [131]:
from langchain.chains import SequentialChain

In [157]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate this review [{review}] into French."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="fr_translation", #Give a name to your output
                     verbose=True
                    )


In [158]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in a short sentence: {fr_translation}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_summary", #give a name to this output
                     verbose=True
                    )


In [159]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate this review summary [{review_summary}] into German"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="de_translation",
                       verbose=True
                      )


In [160]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    """
    Rate the quality of the following translation:
    Original text: {review_summary}
    German translation: {de_translation}
    """
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="translation_rating",
                      verbose=True
                     )


In [161]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translation_rating"],
    verbose=True
)

In [165]:
for review in df.Review[0:3]:
    print(overall_chain(review))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this review [I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.] into French.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Summarize the following review in a short sentence: J'ai commandé un ensemble en taille king. Ma seule critique serait que j'aimerais que le vendeur propose l'ensemble en taille king avec 4 taies d'oreiller. J'ai commandé séparément un pack de deux taies d'oreiller pour en a

**Repeat the above twice for different products or reviews**

## Router Chain

In [166]:
physics_template = """You are a very smart physics professor.
You are great at answering questions about physics in a concise\
and easy to understand manner.
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician.
You are great at answering math questions.
You are so good because you are able to break down
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian.
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods.
You have the ability to think, reflect, debate, discuss and
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication
skills. You are great at answering coding questions.
You are so good because you know how to solve a problem by
describing the solution in imperative steps
that a machine can easily interpret and you know how to
choose a solution that has a good balance between
time complexity and space complexity. 

Here is a question:
{input}"""

In [184]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}

for name, template in prompt_templates.items():
    print(name)
    print(template)
    print()

physics
input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"), additional_kwargs={})]

math
input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, \nanswer the component parts, and then put them togetherto answer the broader question.\n\nHere is a question:\n{input}'), additional_kwargs={})]

History
i

In [185]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [186]:
llm = ChatOpenAI(temperature=0)

In [239]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [243]:
default_prompt = ChatPromptTemplate.from_template("Answer this general question as best as you can: {input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [282]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
1. Given a raw text input to a language model select the model prompt best suited for the input.
2. You will be given the names of the available prompts and a description of what the prompt is best suited for.
3. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.
4. REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR "DEFAULT" if none is well suited to the input text.
5. NEVER set as "destination" a prompt name that is not available in the << CANDIDATE PROMPTS >>, use "DEFAULT" instead.
6. REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string name of the prompt to use or "DEFAULT".
    "next_inputs": string a potentially modified version of the original input
}}}}
```

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [283]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# check output, should be a dict
question = 'give me the theorem of Pythagore'
print(router_chain.invoke(question))

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )




{'input': 'give me the theorem of Pythagore', 'destination': 'math', 'next_inputs': {'input': 'give me the theorem of Pythagoras'}}


In [284]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")
print(response)



> Entering new MultiPromptChain chain...
math: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.
{'input': 'Explain what principle component analysis is using linear algebra', 'text': 'Principal component analysis (PCA) is a technique used in statistics and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. In other words, PCA helps to identify the most important features or components of a dataset.\n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA using linear algebra, the steps typically involve:\n1. Standardizing the data by subtracting the mean and dividing by the standard deviation of each feature.\n2. Computing the covariance matrix of the 

In [285]:
response = chain.run("What is black body radiation?")
print(response)



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics.


In [286]:
response = chain.run("what is 2 + 2")
print(response)



> Entering new MultiPromptChain chain...
math: {'input': 'what is the sum of 2 and 2'}
> Finished chain.
The sum of 2 and 2 is 4.


In [ ]:
# THIS ONE DOES NOT WORK
# it returns me a ValueError: Received invalid destination chain name 'biology'
# the model is hallucinating and inventing prompt names that do no exist

response = chain.run("Why does every cell in our body contain DNA?")
print(response)

In [268]:
response = chain.run("Who holds the fastest marathon time reccord to this day?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'Who holds the fastest marathon time record to this day?'}
> Finished chain.
As of now, the fastest marathon time record is held by Eliud Kipchoge of Kenya, who completed the marathon in a time of 2 hours, 1 minute, and 39 seconds at the 2018 Berlin Marathon.


**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [270]:
response = chain.run("How many muscles and bones is there in the human body?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'How many muscles and bones is there in the human body?'}
> Finished chain.
The human body has 206 bones and over 600 muscles.


In [289]:
response = chain.run("How many muscles and bones is there in the human body?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'How many muscles and bones are there in the human body?'}
> Finished chain.
The human body has over 600 muscles and 206 bones.


In [271]:
response = chain.run("What movie won the most oscars to this day?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'What movie won the most oscars to this day?'}
> Finished chain.
The movie that has won the most Oscars to date is "Ben-Hur" (1959), which won a total of 11 Academy Awards.


In [288]:
response = chain.run("I am super hungry, what should I eat?")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'I am super hungry, what should I eat?'}
> Finished chain.
It depends on your personal preferences and dietary restrictions. Some options could include a balanced meal with protein, vegetables, and carbohydrates, a hearty sandwich or wrap, a bowl of soup or salad, a smoothie or protein shake, or a snack like nuts, fruit, or yogurt. Ultimately, choose something that will satisfy your hunger and provide you with the nutrients you need.
